In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import optuna
import time
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
train_df=pd.read_csv("./energy-anomaly-detection/train.csv")
train_df
unique_ids=train_df.building_id.unique()

df_list=[]

In [3]:
for b_id in unique_ids:
    building_df=train_df.groupby('building_id').get_group(b_id).copy(deep=True)
    building_df['meter_reading']=building_df['meter_reading'].replace(float('nan'),
                                                    building_df['meter_reading'].median())
    building_df.reset_index(drop=True,inplace=True)
    df_list.append(building_df)

print(len(df_list))

200


In [4]:
imputed_train=pd.concat(df_list)

imputed_train
imputed_train.anomaly.value_counts()

anomaly
0    1712198
1      37296
Name: count, dtype: int64

In [5]:
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.preprocessing import StandardScaler

seed = 42
import joblib

In [6]:
def calculate_mz(values):
    median = np.median(values)
    mid_diff = np.abs(values - median)
    epsilon = 1e-9
    mid_median = np.median(mid_diff)
    b_n = 1.486
    sigma_mad = b_n * mid_median + epsilon
    z_scores = mid_diff / sigma_mad

    return z_scores

In [7]:
f1_scores = []
prec_scores = []
recall_scores = []
all_ids = imputed_train.building_id.unique()
for _id in all_ids:
    building = imputed_train[imputed_train['building_id'] == _id].copy(deep=True)
    X_train = building['meter_reading']
    y_train = building['anomaly'].values
    scaler = StandardScaler()
    X_train = scaler.fit_transform(pd.DataFrame(X_train))
    ind_best_f1 = 0
    z_scores = calculate_mz(X_train)
    z_scores = z_scores.reshape(-1)
    for K in range(5, 45, 5):
        K /= 10  # 0.5,1,1.5,2,...
        low = -K
        high = K
        y_pred = np.zeros(len(y_train))
        bool_array = (z_scores > high) | (z_scores < low)
        bool_array = bool_array.reshape(-1)
        y_pred[bool_array] = 1

        score_f1 = f1_score(y_train, y_pred)

        if score_f1 > ind_best_f1:
            score_prec = precision_score(y_train, y_pred)
            score_recall = recall_score(y_train, y_pred)
            ind_best_f1 = score_f1
            ind_best_prec = score_prec
            ind_best_recall = score_recall
    f1_scores.append(ind_best_f1)
    prec_scores.append(ind_best_prec)
    recall_scores.append(ind_best_recall)

In [8]:
print(f1_scores)
print(prec_scores)
print(recall_scores)

[0.044541010320478004, 0.03650190114068441, 0.07648494711147275, 0.08061002178649238, 0.12195121951219512, 0.0724191063174114, 0.11840411840411841, 0.03722943722943723, 0.15243004418262152, 0.4547738693467337, 0.4052863436123348, 0.08055944055944056, 0.6654275092936803, 0.42105263157894735, 0.43243243243243246, 0.657243816254417, 0.7142857142857143, 0.43263757115749524, 0.3641456582633053, 0.21496437054631828, 0.13676633444075303, 0.36731255265374896, 0.1831453218314532, 0.7685589519650655, 0.40572792362768495, 0.019988685649632285, 0.04183266932270916, 0.3008849557522124, 0.11538461538461539, 0.15503875968992248, 0.8036253776435045, 0.14516129032258066, 0.3081761006289308, 0.019313304721030045, 0.5072463768115942, 0.2011527377521614, 0.05156990001754078, 0.2714285714285714, 0.6893939393939394, 0.12935323383084577, 0.05705059203444564, 0.08123791102514506, 0.11042944785276074, 0.02547065337763012, 0.17161500815660685, 0.025870646766169153, 0.12885985748218529, 0.02717391304347826, 0.05

In [9]:
print(np.mean(f1_scores))

print(np.mean(prec_scores))

print(np.mean(recall_scores))

0.3858755425025749
0.46200336801800934
0.6213481696395902
